In [3]:
#需要在同一个目录下建一个叫test01的文件夹，下面有三个文件夹，分别是tmp1，tmp2和result
#tmp1, tmp2用于暂存处理中的图像，result用于存两个视频

#图片结果需要建立一个叫integration的文件夹，下面有三个文件夹，分别是algo1，algo2和intg
#algo1和algo2分别用于存算法1、2的输出，intg用于存整合的后图像

#网页的两个视频在test01/result/下，图片在integration/intg/下

import cv2
import numpy as np
import time
import glob
import matplotlib.pyplot as plt
import os

img3 = integration("五矿1.mp4")


In [ ]:
def denoise(input_image):
    input_image = cv2.threshold(input_image, 254, 255, cv2.THRESH_BINARY)[1]
    input_image_comp = cv2.bitwise_not(input_image)  # could just use 255-img

    kernel1 = np.array([[0, 0, 0],
                        [0, 1, 0],
                        [0, 0, 0]], np.uint8)
    kernel2 = np.array([[1, 1, 1],
                        [1, 0, 1],
                        [1, 1, 1]], np.uint8)

    hitormiss1 = cv2.morphologyEx(input_image, cv2.MORPH_ERODE, kernel1)
    hitormiss2 = cv2.morphologyEx(input_image_comp, cv2.MORPH_ERODE, kernel2)
    hitormiss = cv2.bitwise_and(hitormiss1, hitormiss2)
    output_img = cv2.addWeighted(input_image,1,hitormiss,-1,0)
    return(output_img)

In [14]:
def algo1(vv):
    print(vv)
    files = glob.glob('test01/tmp1/*')
    for f in files:
        os.remove(f)
    files = glob.glob('test01/tmp2/*')
    for f in files:
        os.remove(f)
    files = glob.glob('test01/*.jpg')
    for f in files:
        os.remove(f)
    v_path = "视频/{}".format(vv)
    cap = cv2.VideoCapture(v_path)
    frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    print("帧数:{}".format(frame_count))
    for i in range(int(frame_count)):
        _,img = cap.read(i)
        grayImage = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        kernelx = np.array([[1,1,1],[0,0,0],[-1,-1,-1]],dtype=int)
        kernely = np.array([[-1,0,1],[-1,0,1],[-1,0,1]],dtype=int)
        x = cv2.filter2D(grayImage, cv2.CV_16S, kernelx)
        y = cv2.filter2D(grayImage, cv2.CV_16S, kernely)
        absX = cv2.convertScaleAbs(x) 
        absY = cv2.convertScaleAbs(y) 
        Prewitt = cv2.addWeighted(absX,0.5,absY,0.5,0)
        cv2.imwrite("test01/tmp1/{:05d}.jpg".format(i), Prewitt)
    img_array = []
    for i in range(int(frame_count)): 
        filename = 'test01/tmp1/{:05d}.jpg'.format(i)
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width,height)
        img_array.append(img)
    out = cv2.VideoWriter('test01/result/algo1_processed1.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 10, size)
    for i in range(len(img_array)):
        out.write(img_array[i])
    out.release()
    video_path = 'test01/result/algo1_processed1.mp4'
    cap = cv2.VideoCapture(video_path)
    width =int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height =int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    one_frame = np.zeros((height,width),dtype=np.uint8)
    two_frame = np.zeros((height,width),dtype=np.uint8)
    three_frame = np.zeros((height,width),dtype=np.uint8)
    for k in range(int(frame_count)):
        ret,frame = cap.read()
        frame_gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        if not ret:
            break
        one_frame,two_frame,three_frame = two_frame,three_frame,frame_gray
        abs1 = cv2.absdiff(one_frame,two_frame)
        _,thresh1 = cv2.threshold(abs1,15,555,cv2.THRESH_BINARY)
        abs2 =cv2.absdiff(two_frame,three_frame)
        _,thresh2 =cv2.threshold(abs2,15,555,cv2.THRESH_BINARY)
        binary =cv2.bitwise_and(thresh1,thresh2)
        cv2.imwrite("test01/tmp2/{:05d}.jpg".format(k),binary)
        if cv2.waitKey(50)&0xFF==ord("q"):
            break
    cap.release()
    cv2.destroyAllWindows()
    
    II = cv2.imread("test01/tmp2/00001.jpg")
    for k in range(int(frame_count)):
        img = cv2.imread("test01/tmp2/{:05d}.jpg".format(k))
        II = II + img
    cv2.imwrite("integration/algo1/algo1_processed_{}.jpg".format(vv), II)
    return(II)

In [17]:
def algo2(vv):    
    print(vv)
    files = glob.glob('test01/tmp1/*')
    for f in files:
        os.remove(f)
    files = glob.glob('test01/tmp2/*')
    for f in files:
        os.remove(f)
    files = glob.glob('test01/*.jpg')
    for f in files:
        os.remove(f)

    v_path = "视频/{}".format(vv)
    cap = cv2.VideoCapture(v_path)
    frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    print("帧数:{}".format(frame_count))
    

    for i in range(int(frame_count)):
        _,img = cap.read(i)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        image = img.copy()
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        gaussian_3 = cv2.GaussianBlur(image, (21, 21), cv2.BORDER_DEFAULT)
        img = cv2.addWeighted(image, 6, gaussian_3, -5.0, 0)
        fgamma = 2
        img = np.uint8(np.power((np.array(img)/255.0),fgamma)*255.0)
        img = cv2.normalize(img, img, 0, 255, cv2.NORM_MINMAX)
        img = cv2.convertScaleAbs(img, img)

        cv2.imwrite("test01/{}.jpg".format(i), img)
        

    for i in range(int(frame_count)):
        img = cv2.imread('test01/{}.jpg'.format(i))
        grayImage = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        kernelx = np.array([[1,1,1],[0,0,0],[-1,-1,-1]],dtype=int)
        kernely = np.array([[-1,0,1],[-1,0,1],[-1,0,1]],dtype=int)
        x = cv2.filter2D(grayImage, cv2.CV_16S, kernelx)
        y = cv2.filter2D(grayImage, cv2.CV_16S, kernely)
        absX = cv2.convertScaleAbs(x) 
        absY = cv2.convertScaleAbs(y) 
        Prewitt = cv2.addWeighted(absX,0.5,absY,0.5,0)
        
        cv2.imwrite("test01/tmp1/{:05d}.jpg".format(i), Prewitt)
    
    img_array = []
    for i in range(int(frame_count)):
        filename = 'test01/tmp1/{:05d}.jpg'.format(i)
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width,height)
        img_array.append(img)
    out = cv2.VideoWriter('test01/result/algo2_processed.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 10, size)
    for i in range(len(img_array)):
        out.write(img_array[i])
    out.release()
    
    video_path = 'test01/result/algo2_processed.mp4'
    cap = cv2.VideoCapture(video_path)

    width =int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height =int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    one_frame = np.zeros((height,width),dtype=np.uint8)
    two_frame = np.zeros((height,width),dtype=np.uint8)
    three_frame = np.zeros((height,width),dtype=np.uint8)

    for k in range(int(frame_count)):
        ret,frame = cap.read()
        frame_gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        if not ret:
            break
        one_frame,two_frame,three_frame = two_frame,three_frame,frame_gray
        abs1 = cv2.absdiff(one_frame,two_frame)
        _,thresh1 = cv2.threshold(abs1,15,555,cv2.THRESH_BINARY)
        abs2 =cv2.absdiff(two_frame,three_frame)
        _,thresh2 =cv2.threshold(abs2,15,555,cv2.THRESH_BINARY)
        binary =cv2.bitwise_and(thresh1,thresh2)
        cv2.imwrite("test01/tmp2/{:05d}.jpg".format(k),binary)
        if cv2.waitKey(50)&0xFF==ord("q"):
            break
    cap.release()
    cv2.destroyAllWindows()

    II = cv2.imread("test01/tmp2/00001.jpg")
    for k in range(int(frame_count)):
        img = cv2.imread("test01/tmp2/{:05d}.jpg".format(k))
        II = II + img
    cv2.imwrite("integration/algo2/algo2_processed_{}.jpg".format(vv), II)

    return(II)

In [21]:
def integration(vv):
    p1 = algo1(vv)
    p2 = algo2(vv)
    p1 = cv2.threshold(p1, 240, 255, cv2.THRESH_BINARY)[1]
    p2 = cv2.threshold(p2, 250, 255, cv2.THRESH_BINARY)[1]
    p3 = cv2.bitwise_or(p1,p2)
    p3 = denoise(p3)
    cv2.imwrite("integration/intg/intg_{}.jpg".format(vv), p3)
    return(p3)